# Compute a ratio of variances test using the F-statistic
The F-statistic is just the ratio of two variances.  If we know the degrees of freedom (DOF) of the two variances, we can compute the probability that the top one exeeds the lower one.
First we will input two data sets and compute things.
Then we will address the case where we have the ratio of variances and the DOF and we want to compute the probability.

In [52]:
import numpy as np
import scipy.stats

#define F-test function where we input two time series x and y
def f_test(x, y):
    x = np.array(x)
    y = np.array(y)
    f = np.var(x, ddof=1)/np.var(y, ddof=1) #calculate F test statistic 
    dfn = x.size-1 #define degrees of freedom numerator 
    dfd = y.size-1 #define degrees of freedom denominator 
    p = 1-scipy.stats.f.cdf(f, dfn, dfd) #find p-value of F test statistic 
    return f, p, dfn, dfd


In [53]:
# Import text file containing test data from Homework 2
X = np.loadtxt('Data_HW_2.txt')
N = np.shape(X)   # what is the shape of the imported data set (500,2)
N0=N[0]
#print(N)
#t = np.arrange(1,N,1)
t = np.linspace(1,N0,N0)   # Make a linear series to be like the time
x = X[:,0]
y = X[:,1]
f, p, dfn, dfd = f_test(y,x)
print('F=statistic = ',f,', p-value = ',p,', DOF num = ',dfn,', DOF den = ',dfd)

F=statistic =  18.924136315982164 , p-value =  1.1102230246251565e-16 , DOF num =  499 , DOF den =  499


### Interpretation
What the above means is that the F-statistic is 18.9 with a tiny p-value, meaning that the variances are highly significantly different.

## Application to Spectral Analysis
Next, lets make a function that takes the F value and the DOF and gives a significance level.  This would be useful if we had the ratio of variances, say from spectral analysis, and we knew the dof of the spectral estimate and our null hypothesis.
In the following example suppose that the spectral peak exceeds its null hypothesis by a factor of 2, the dof of the spectral estimate is 20 and the dof of the null hypothesis is 100.

In [62]:
F = 2
dfn = 23
dfd = 100
p = 1-scipy.stats.f.cdf(F, dfn, dfd)
print('F=statistic = ',F,', p-value = ',p,', DOF numerator = ',dfn,', DOF denominator = ',dfd)

F=statistic =  2 , p-value =  0.010057134790730893 , DOF numerator =  23 , DOF denominator =  100


### What we really want is the needed F-statistic to pass a particular confidence level with known DOF on top and bottom
Let's try that next

In [55]:
#  define get_F function, which just iterates
# p is the desired probability level, dfn and dfd are the DOF of the numerator and denomiator, F0 is a starting value
# F0 is a starting value, df is the F increment, and itmx is the maximum number of iterates
def get_F(p, dfn, dfd, F0, df, itmx):
    success = False
    for i in range(itmx):
        F=F0+df*i
        p0 = 1-scipy.stats.f.cdf(F, dfn, dfd)
        if p0<p:
            success = True
            break

    return F, success

In [56]:
# ok lets test out our function
p=0.01
dfn=20
dfd=100
F0=1.0
df=0.05
itmx=100
F, success = get_F(p, dfn, dfd, F0, df, itmx)
print('F = ',F,' Success ' ,success)
dfd =20
F, success = get_F(p, dfn, dfd, F0, df, itmx)
print('F = ',F,' Success ' ,success)
F = get_F(p, dfn, dfd, F0, df, itmx)
print('F = ',F)

F =  2.1  Success  True
F =  2.95  Success  True
F =  (2.95, True)


In [57]:
#  define get_F function, which just iterates  % this version assumes an F0, df and itmx
# p is the desired probability level, dfn and dfd are the DOF of the numerator and denomiator, F0 is a starting value
# F0 is a starting value, df is the F increment, and itmx is the maximum number of iterates
def get_F2(p, dfn, dfd):
    F0=1.0
    df=0.05
    itmx=100  # this produces a range of F from 1.0 to 6.0  If you are outside this range your sample is too small
    success = False
    for i in range(itmx):
        F=F0+df*i
        p0 = 1-scipy.stats.f.cdf(F, dfn, dfd)
        if p0<p:
            success = True
            break

    return F, success

In [58]:
# ok lets test out our function
p=0.01
dfn=25  # this example shows that if you want to show p=0.01 with F=2.0, you need 25 DOF in the numerator if you assume 100 dof for the denominator
dfd=100  # once the DOF of the denominator exceeds about 100, the critical F-statistic becomes insensitive to further increases in dfd

F, success = get_F2(p, dfn, dfd)
print('F = ',F,' Success ' ,success)
dfd =20
F, success = get_F(p, dfn, dfd, F0, df, itmx)
print('F = ',F,' Success ' ,success)
F = get_F(p, dfn, dfd, F0, df, itmx)
print('F = ',F)

F =  2.0  Success  True
F =  2.85  Success  True
F =  (2.85, True)


### Final Comment
Must be an easier way than get_F2 to do this, but it works OK